In [25]:
"""
Template for week 14 project in Data Visualization

Load binary CT data and plot the contours of the resulting volume
http://graphics.cs.ucdavis.edu/~okreylos/PhDStudies/Spring2000/ECS277/DataSets.html
"""

import struct
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
import skimage

In [26]:
# Note that file names are caps-sensitive on Unix
PLOTS_PATH = "plots/"
DATA_PATH = "data/"
SIMPLE = DATA_PATH + "simple.vol"
C60_64 = DATA_PATH + "C60_64.vol"
C60_128 = DATA_PATH + "C60_128.vol"
FOOT = DATA_PATH + "Foot.vol"
SKULL = DATA_PATH + "Skull.vol"

VOLS = {}

# Custom colorscale for CT volumes
PL_BONE = [
    [0.0, 'rgb(0, 0, 0)'],
    [0.05, 'rgb(10, 10, 14)'],
    [0.1, 'rgb(21, 21, 30)'],
    [0.15, 'rgb(33, 33, 46)'],
    [0.2, 'rgb(44, 44, 62)'],
    [0.25, 'rgb(56, 55, 77)'],
    [0.3, 'rgb(66, 66, 92)'],
    [0.35, 'rgb(77, 77, 108)'],
    [0.4, 'rgb(89, 92, 121)'],
    [0.45, 'rgb(100, 107, 132)'],
    [0.5, 'rgb(112, 123, 143)'],
    [0.55, 'rgb(122, 137, 154)'],
    [0.6, 'rgb(133, 153, 165)'],
    [0.65, 'rgb(145, 169, 177)'],
    [0.7, 'rgb(156, 184, 188)'],
    [0.75, 'rgb(168, 199, 199)'],
    [0.8, 'rgb(185, 210, 210)'],
    [0.85, 'rgb(203, 221, 221)'],
    [0.9, 'rgb(220, 233, 233)'],
    [0.95, 'rgb(238, 244, 244)'],
    [1.0, 'rgb(255, 255, 255)']
]


In [27]:
#########################################################################
# Part 1 - Implement Volume class and make_volume()

class Volume:
    """
    Container for CT volume data
    """
    
    def __init__(self, grid_values, grid_extents=(1, 1, 1)):
        """
        Input: 3D numpy array grid_values, optional tuple grid_extents
        that contains extent of grid in each dimension
        
        Ouput: Object storing grid of scalar data
        
        """
        self._data = grid_values
        self._extents = grid_extents
        
    
    # Implement during Part 3
    def plot_volume_contour(self, val, title="Contour plot of volume", plot_edges=True):
        """
        Input: Volume object self, number val, optional string title
        
        Output: plotly figure corresponding to contour plot of volume using Marching Cubes
        with specified value val.  Use grid extents to set aspect ratio.
        Also writes HTML to PLOTS_PATH + title + ".html".
        """
        verts, faces = skimage.measure.marching_cubes_classic(self._data, level=val)

        fig = ff.create_trisurf(
            x=verts[:, 2], y=verts[:, 1], z=verts[:, 0],
            simplices=faces,
            title=title,
            plot_edges=plot_edges,
            show_colorbar=False,
        )

        fig.update_layout(
            scene=dict(
                aspectmode='manual',
                aspectratio=dict(
                    x=1,
                    y=self._extents[1] / self._extents[2],
                    z=self._extents[0] / self._extents[2],
                )
            ),
        )

        # Save the figure
        pio.write_html(fig, file=PLOTS_PATH + title + ".html")
        return fig

    # Implement during Part 4
    def plot_volume_slice(self, title):
        """
        Input: Volume object self, optional string title
    
        Output: plotly figure corresponding to 3D slices of volume 
        perpendicular to z-axis. The vertical position of this slice 
        should be controlled by buttons and a slider
        Also writes HTML to PLOTS_PATH + title + ".html".
        """   
        volume_data = self._data
        total_slices = volume_data.shape[0]
        cmin = np.min(volume_data)
        cmax = np.max(volume_data)

        # Setup animation frames
        frames = [
            go.Frame(
                data=[go.Surface(
                    z=slice_index * np.ones(volume_data[0].shape), 
                    surfacecolor=np.flipud(volume_data[slice_index]),
                    colorscale=PL_BONE,  
                    cmin=cmin,
                    cmax=cmax,
                )], 
                name=str(slice_index)
            ) for slice_index in range(total_slices)
        ]

        # Initial surface display
        initial_display = go.Surface(
            z=np.zeros(volume_data[0].shape),
            surfacecolor=np.flipud(volume_data[0]),
            colorscale=PL_BONE, 
            cmin=cmin,
            cmax=cmax,
            colorbar=dict(thickness=20, ticklen=4)
        )

        # Create the figure with initial display and frames
        fig = go.Figure(
            data=[initial_display],
            frames=frames,
            layout=go.Layout(
                title=title,
                width=600,
                height=600,
                scene=dict(
                    zaxis=dict(range=[0, total_slices - 1], autorange=False),
                    aspectratio=dict(
                        x=1,
                        y=self._extents[1] / self._extents[2],
                        z=self._extents[0] / self._extents[2]
                    )
                )
            )
        )

        # Slider configuration
        sliders = [{
            "steps": [
                {"args": [[frame.name], 
                          {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                 "label": str(index),
                 "method": "animate"}
                for index, frame in enumerate(frames)
            ],
            "pad": {"b": 10, "t": 60},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
        }]
        fig.update_layout(sliders=sliders)


        # Save the figure as an HTML file
        pio.write_html(fig, file=PLOTS_PATH + title + ".html")
        return fig

 

In [28]:
def test_volume():
    """ Test the Volume class initializer """
    
    Volume(np.array([[[0]]]))
    Volume(np.array([[[1, 1], [1, 1]], [[2, 2], [2, 2]], [[3, 3], [3, 3]]]), (3, 2, 2))
    
test_volume()

In [29]:
def make_volume(z_coords, y_coords, x_coords, grid_fun):
    """
    Input: Numpy arrays z_coords, y_coords, x_coords,
    function grid_fun that takes 3 scalar parameters

    Output: Volume object whose grid values as grid_fun(z, y, x)
    taken at points of the grid defined by the coordinate arrays
    """
    
    z_val, y_val, x_val = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')

    grid_values = grid_fun(z_val, y_val, x_val)

    z_extent = np.max(z_coords) - np.min(z_coords)
    y_extent = np.max(y_coords) - np.min(y_coords)
    x_extent = np.max(x_coords) - np.min(x_coords)
    extents = (z_extent, y_extent, x_extent)

    return Volume(grid_values, extents)

In [30]:
def test_make_volume():
    """ Test make_volume function """

    z_values = np.linspace(-3, 3, 7)
    y_values = np.linspace(-2, 2, 5)
    x_values = np.linspace(-1, 1, 5)
    grid_fun = lambda z, y, x: x + y + z
    VOLS["linear"] = make_volume(z_values, y_values, x_values, grid_fun)
    
    z_values = np.linspace(-3, 3, 13)
    y_values = np.linspace(-3, 3, 13)
    x_values = np.linspace(-3, 3, 13)
    grid_fun = lambda z, y, x: z ** 2 + y ** 2 + x ** 2
    VOLS["spherical"] = make_volume(z_values, y_values, x_values, grid_fun)
    
test_make_volume()

In [31]:
############################################################################
# Part 2 - Read binary CT data from a file and create a Volume object


def read_volume(vol_name):
    """
    Input: String vol_name 
    
    Output: Volume object read from binary-encoded CT volume data in given file
    
    NOTE: Use struct module - https://docs.python.org/3/library/struct.html
    """
    with open(vol_name, 'rb') as file:
        header_format = ">3i4f"  
        header_size = struct.calcsize(header_format)
        header_data = file.read(header_size)
        unpacked_header = struct.unpack(header_format, header_data)
    
        dims = unpacked_header[:3]  
        extents = unpacked_header[4:]  
        
        num_voxels = dims[0] * dims[1] * dims[2]
        vol_data_format = ">" + str(num_voxels) + "B"
        vol_data = file.read(num_voxels)
        volume_values = np.array(struct.unpack(vol_data_format, vol_data), dtype=np.uint8)
        reshaped_vol_data = np.reshape(volume_values, dims)

        return Volume(reshaped_vol_data, extents)

    

In [32]:
def test_read_volume():
    """ Test read_volume() """
    
    # Simple test case to catch transpose errors
    simple_vol = read_volume(SIMPLE)
    print(simple_vol._extents, "\n\n", simple_vol._data)
    
    VOLS["C60_64"] = read_volume(C60_64)    
    VOLS["FOOT"] = read_volume(FOOT)
    VOLS["SKULL"] = read_volume(SKULL)
    
test_read_volume()

(2.0, 3.0, 4.0) 

 [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


Correct output
~~~~
(2.0, 3.0, 4.0) 

 [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
~~~~

In [9]:
################################################################################
# Part 3 - Add contour_volume method to the Volume class

def test_plot_volume_contour():
    """ Test plot_volume_contour() """
    
    VOLS["linear"].plot_volume_contour(0, "Contour plot of linear function")    
    VOLS["spherical"].plot_volume_contour(4, "Contour plot of spherical function", plot_edges=False)
    
    VOLS["C60_64"].plot_volume_contour(127, "Contour plot of C60")
    VOLS["FOOT"].plot_volume_contour(127, "Contour plot of human foot")
    VOLS["SKULL"].plot_volume_contour(127, "Contour plot of human skull")
    
test_plot_volume_contour()

## Links to the 5 triangular meshes produced by `test_plot_volume_contour()`

* [Contour plot of linear function](plots/Contour%20plot%20of%20linear%20function.html)
* [Contour plot of spherical function](plots/Contour%20plot%20of%20spherical%20function.html)
* [Contour plot of C60](plots/Contour%20plot%20of%20C60.html)
* [Contour plot of human foot](plots/Contour%20plot%20of%20human%20foot.html)
* [Contour plot of human skull](plots/Contour%20plot%20of%20human%20skull.html)

In [33]:
################################################################################
# Part 4 - Add plot_volume_slice method to the Volume class

def test_plot_volume_slice():
    """ Test plot_volume_slice() """
    
    VOLS["linear"].plot_volume_slice("Horizontal slices of linear function")    
    VOLS["spherical"].plot_volume_slice("Horizontal slices of spherical function")
    
    VOLS["C60_64"].plot_volume_slice("Horizontal slices of C60")
    VOLS["FOOT"].plot_volume_slice("Horizontal slices of human foot")
    VOLS["SKULL"].plot_volume_slice("Horizontal slices of human skull")
    
test_plot_volume_slice()

### Links to the horizontal slicing of the 5 volumes produced by `test_plot_volume_slice()`
* [Horizontal slices of linear function](plots/Horizontal%20slices%20of%20linear%20function.html)
* [Horizontal slices of spherical function](plots/Horizontal%20slices%20of%20spherical%20function.html)
* [Horizontal slices of C60_64](plots/Horizontal%20slices%20of%20C60.html)
* [Horizontal slices of human foot](plots/Horizontal%20slices%20of%20human%20foot.html)
* [Horizontal slices of human skull](plots/Horizontal%20slices%20of%20human%20skull.html)